In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory

print("Using TensorFlow version:", tf.__version__)

# Pour reproduire un comportement stable (optionnel)
tf.random.set_seed(42)

/Users/emmacoco/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Using TensorFlow version: 2.18.0


## 1. Chargement du dataset

In [3]:
# Chemin vers le dossier qui contient les deux sous-dossiers : 'good' et 'defect'
DATA_DIR = "screw_dataset"  # À adapter

BATCH_SIZE = 32
IMG_SIZE = (224, 224)

# Charger toutes les images d'un seul tenant
dataset = image_dataset_from_directory(
    DATA_DIR,
    labels='inferred',
    label_mode='categorical',  # Pour un problème multi-classes, ici on suppose 2 classes (good, defect)
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True  # Mélange les données
)

# Afficher les classes détectées
class_names = dataset.class_names
print("Classes détectées :", class_names)

# Optionnel : Normalisation et mise en cache
AUTOTUNE = tf.data.AUTOTUNE
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalisation simple
    return image, label

dataset = dataset.map(preprocess).cache().prefetch(buffer_size=AUTOTUNE)

Found 1152 files belonging to 2 classes.
Classes détectées : ['bad', 'good']


## 2. Définition d'un modèle simple
On définit un modèle CNN

In [4]:
num_classes = len(class_names)

model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['accuracy']
)

model.summary()

/Users/emmacoco/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 93312)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     5,972,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,977,250 (22.80 MB)

 Trainable params: 5,977,250 (22.80 MB)

 Non-trainable params: 0 (0.00 B)

## 3. Entraînement du modèle

In [ ]:
EPOCHS = 1000  # Nombre d'époques d'entraînement

history = model.fit(
    dataset,                 # Entraînement ET test sur le même dataset
    epochs=EPOCHS,
    verbose=1
)

## 4. Évaluation sur le même dataset

In [ ]:
loss, accuracy = model.evaluate(dataset, verbose=0)
print(f"Loss sur le dataset complet : {loss:.4f}")
print(f"Accuracy sur le dataset complet : {accuracy:.4f}")